In [ ]:
from extract_lcomp import *
from graphliker import * # Has all the nice shit imported already
from copy import deepcopy
import time
extra_params["max_greed"] = 20
#import random
#random.seed(3)

In [ ]:
circ = zx.generate.CNOT_HAD_PHASE_circuit(10, 800)
g = circ.to_graph()
zx.simplify.full_reduce(g, quiet = True)
g.normalize()
zx.draw(g)

In [ ]:
strats = [(zx.extract_circuit, "PyZX extract"), (extract_circuit, "Injected Extract")]
prev_out = None
first_two_qubit_gatecount = None
first_time = None

compare = True

for (func, name) in strats:
    t_start = time.perf_counter()
    out = func(deepcopy(g))
    t_end = time.perf_counter()
    out = zx.optimize.basic_optimization(out.to_basic_gates(), do_swaps = True).to_basic_gates()
    delta_time = t_end - t_start
    print(f"Strategy {name} ({delta_time:0.4f}s):")
    print(out.stats())
    zx.draw(out)

    if first_two_qubit_gatecount == None:
        first_two_qubit_gatecount = out.twoqubitcount()
        first_time = delta_time
    else:
        print("")
        two_qubit_gatecount = out.twoqubitcount()
        percentage = int(100 * two_qubit_gatecount / first_two_qubit_gatecount) - 100
        print(f"GATE COUNT CHANGE: {percentage:+4}% ({first_two_qubit_gatecount} ↦ {two_qubit_gatecount})")
        percentage = int(100 * delta_time / first_time)
        print(f"      PERFORMANCE: {percentage:4}% ({first_time:0.4f}s ↦ {delta_time:0.4f}s)")
        print("")

    if compare and prev_out != None:
        print("Starting comparison")
        if not zx.compare_tensors(out, prev_out):
            raise ValueError("welp")
        print("Finished comparison: no problems")
    prev_out = out